In [1]:
import cfbd
import pandas as pd
import numpy as np


config = cfbd.Configuration()
config.api_key['Authorization'] = 'ti+f5GzjSdgTT5Lc1m5PcOE4kcRZrs327v2VUYK8sUo2sSIEvnZZQ08FUsINUCYL'
config.api_key_prefix['Authorization'] = 'Bearer'

api_instance = cfbd.PlaysApi(cfbd.ApiClient(config))

dataframe = pd.DataFrame()

big_ten_teams = ["Minnesota", "Wisconsin", "Ohio State", "Penn State", "Michigan", "Michigan State", "Nebraska", "Northwestern", "Illinois", "Indiana", "Iowa", "Maryland", "Purdue", "Rutgers"]

for team in big_ten_teams:
    print(team)
    config = cfbd.Configuration()
    config.api_key['Authorization'] = 'ti+f5GzjSdgTT5Lc1m5PcOE4kcRZrs327v2VUYK8sUo2sSIEvnZZQ08FUsINUCYL'
    config.api_key_prefix['Authorization'] = 'Bearer'

    api_instance = cfbd.PlaysApi(cfbd.ApiClient(config))
    for year in range(2017,2023):
        print(year)
        for i in range(1,14):
            plays_object = api_instance.get_plays(year, week=i, team=team)

            plays_dict = []

            for play in plays_object:
                plays_dict.append(vars(play))

            for play in plays_dict: 
                time = vars(play["_clock"])
                play["minutes"] = time["_minutes"]
                play["seconds"] = time['_seconds']

            frame = pd.DataFrame(plays_dict) 
            dataframe = pd.concat([dataframe, frame])
    dataframe.to_csv(f"epa_after_{team}.csv")

dataframe.head()

Minnesota
2017
2018
2019
2020
2021
2022
Wisconsin
2017


KeyboardInterrupt: 

In [ ]:
import cfbd

dataframe = pd.read_csv('epa_after_Iowa.csv')

big_ten_teams = ["Maryland", "Purdue", "Rutgers"]

for team in big_ten_teams:
    print(team)
    config = cfbd.Configuration()
    config.api_key['Authorization'] = 'ti+f5GzjSdgTT5Lc1m5PcOE4kcRZrs327v2VUYK8sUo2sSIEvnZZQ08FUsINUCYL'
    config.api_key_prefix['Authorization'] = 'Bearer'

    api_instance = cfbd.PlaysApi(cfbd.ApiClient(config))
    for year in range(2017,2023):
        print(year)
        for i in range(1,14):
            plays_object = api_instance.get_plays(year, week=i, team=team)

            plays_dict = []

            for play in plays_object:
                plays_dict.append(vars(play))

            for play in plays_dict: 
                time = vars(play["_clock"])
                play["minutes"] = time["_minutes"]
                play["seconds"] = time['_seconds']

            frame = pd.DataFrame(plays_dict) 
            dataframe = pd.concat([dataframe, frame])
    dataframe.to_csv(f"epa_after_{team}.csv")

dataframe.head()

Maryland
2017
2018
2019
2020
2021
2022
Purdue
2017
2018
2019
2020
2021
2022
Rutgers
2017
2018
2019
2020
2021
2022


,Unnamed: 0,_configuration,_id,_drive_id,_game_id,_drive_number,_play_number,_offense,_offense_conference,_offense_score,...,_distance,_yards_gained,_scoring,_play_type,_play_text,_ppa,_wallclock,discriminator,minutes,seconds
0,0.0,<cfbd.configuration.Configuration object at 0x...,400935230101849903,4009352301,400935230,1,1,Minnesota,Big Ten,0,...,10,0,False,Kickoff,Ryan Santoso kickoff for 65 yds for a touchback,NaN,NaN,NaN,15,0
1,1.0,<cfbd.configuration.Configuration object at 0x...,400935230101849904,4009352301,400935230,1,2,Buffalo,Mid-American,0,...,10,2,False,Rush,Tyree Jackson run for 2 yds to the Buff 27,-0.452604,NaN,NaN,15,0
2,2.0,<cfbd.configuration.Configuration object at 0x...,400935230101849905,4009352301,400935230,1,3,Buffalo,Mid-American,0,...,8,4,False,Pass Reception,Tyree Jackson pass complete to Jacob Martinez ...,-0.082129,NaN,NaN,15,0
3,3.0,<cfbd.configuration.Configuration object at 0x...,400935230101849906,4009352301,400935230,1,4,Buffalo,Mid-American,0,...,4,-10,False,Penalty,"JACKSON, Tyree rush for 12 yards to the UB43 (...",NaN,NaN,NaN,15,0
4,4.0,<cfbd.configuration.Configuration object at 0x...,400935230101849907,4009352301,400935230,1,5,Buffalo,Mid-American,0,...,14,0,False,Pass Incompletion,Tyree Jackson pass incomplete,-0.078567,NaN,NaN,15,0


In [ ]:
dataframe.to_csv('big_ten_pbp.csv')

In [1]:
import pandas as pd
import numpy as np

dataframe = pd.read_csv('big_ten_pbp.csv')

dataframe["soccer_time"] = dataframe['_period'] * ( 16 - (dataframe['minutes'] + (dataframe['seconds']/60)).round(0)).astype("int32")

soccer_time = range(1,64, 3)
intervals = []
conditions = []

for i in soccer_time:
    if i > 60:
        intervals.append(f"{i}")
        conditions.append( (dataframe["soccer_time"] == i) )
    if i == 58:
        intervals.append(f"{i}-{i+2}")
        conditions.append( (dataframe["soccer_time"] >= i) & (dataframe["soccer_time"] <= (i+2)))
    else:
        intervals.append(f"{i}-{i+3}")
        conditions.append( (dataframe["soccer_time"] >= i) & (dataframe["soccer_time"] <= (i+3)))



dataframe["time_intervals"] = np.select(conditions, intervals)
dataframe["score_differential"] = dataframe["_offense_score"] - dataframe["_defense_score"]
dataframe = dataframe[dataframe["_offense_conference"] == "Big Ten"]

In [2]:
field_pos_range = range(1,100,5)

intervals = []
conditions = []

for i in field_pos_range:
    intervals.append(f"{i}-{i+4}")
    conditions.append( (dataframe["_yard_line"] >= i) & (dataframe["_yard_line"] <= (i+4)))

dataframe["field_position_intervals"] = np.select(conditions, intervals)
dataframe["field_position_intervals"].unique()

array(['31-35', '46-50', '51-55', '36-40', '21-25', '26-30', '41-45',
       '61-65', '66-70', '71-75', '81-85', '86-90', '91-95', '96-100',
       '16-20', '11-15', '0', '6-10', '56-60', '76-80', '1-5'],
      dtype=object)

In [3]:
conditions = [
    (dataframe["_play_type"] == "Field Goal Missed"),
    (dataframe["_play_type"] == "Field Goal Good"),
    (dataframe["_play_type"] == "Passing Touchdown") | (dataframe["_play_type"] == "Rusing Touchdown"),
    (dataframe["_play_type"] == "Interception Return Touchdown") 
    | (dataframe["_play_type"] == "Fumble Return Touchdown") 
    | (dataframe["_play_type"] == "Punt Return Touchdown") 
    | (dataframe["_play_type"] == "Blocked Punt Touchdown")
    | (dataframe["_play_type"] == "Missed Field Goal Return Touchdown")
    | (dataframe["_play_type"] == "Blocked Field Goal Touchdown"),
    (dataframe["_play_type"] == "Safety"),
]

values = [0, 3, 7, -7, 2]

dataframe["points_scored"] = np.select(conditions, values)

dataframe = dataframe[( dataframe["_down"] > 0) ]
dataframe = dataframe[(dataframe["_distance"] > 0)]
dataframe = dataframe[(dataframe["time_intervals"] != '0')]

dataframe[['time_lower', 'time_upper']] = dataframe.time_intervals.str.split('-', expand=True).astype('int')
dataframe[['field_position_lower', 'field_position_upper']] = dataframe.time_intervals.str.split('-', expand=True).astype('int')

dataframe["points_scored"].unique()

array([ 0,  7,  3, -7,  2])

In [4]:
grouped = dataframe.groupby(["_down", "_distance", "time_intervals", "field_position_intervals", "score_differential"])["points_scored"].agg(["sum","count","mean","median"]).reset_index()
grouped[['time_lower', 'time_upper']] = grouped.time_intervals.str.split('-', expand=True).astype('int')
grouped[['field_position_lower', 'field_position_upper']] = grouped.time_intervals.str.split('-', expand=True).astype('int')
grouped.head()

,_down,_distance,time_intervals,field_position_intervals,score_differential,sum,count,mean,median,time_lower,time_upper,field_position_lower,field_position_upper
0,1,1,1-4,1-5,-25,0,2,0.0,0.0,1,4,1,4
1,1,1,1-4,1-5,-7,0,4,0.0,0.0,1,4,1,4
2,1,1,1-4,1-5,0,0,2,0.0,0.0,1,4,1,4
3,1,1,1-4,1-5,3,0,2,0.0,0.0,1,4,1,4
4,1,1,1-4,1-5,7,0,2,0.0,0.0,1,4,1,4


In [5]:
# this cell is placing xP values in the right rows and then subtracting the points scored from the xp to get the xP added for the play.
# I wanted to see the average difference between my xPa model compared to the College football data's version of it
intervals = []
conditions = []

for i in range(len(grouped)):
    intervals.append(grouped["mean"][i])
    conditions.append( (dataframe["_down"] == grouped["_down"][i]) 
                        & (dataframe["_distance"] == grouped['_distance'][i]) 
                        & (dataframe['time_lower'] >= grouped['time_lower'][i])
                        & (dataframe['time_upper'] <= grouped['time_upper'][i])
                        & (dataframe['field_position_lower'] >= grouped['field_position_lower'][i])
                        & (dataframe['field_position_upper'] <= grouped['field_position_upper'][i])
                        & (dataframe['score_differential'] == grouped['score_differential'][i]) )

dataframe["xP"] = np.select(conditions, intervals)
dataframe['xPa'] = dataframe["points_scored"] - dataframe['xP']

epa_diff = dataframe['xPa'] - dataframe['_ppa']
epa_diff.mean()

-0.1459476281652915

In [6]:
# This cell is pulling and calculating the probability a first down is acheived by down and distance
# This will be used in the model making process to gauge how probable it is to make a first down

dataframe['got_first_down'] = dataframe['_yards_gained'] >= dataframe["_distance"]
dataframe['got_first_down'].unique()

first_down_prob = dataframe.groupby(['_down', '_distance'])['got_first_down'].agg(['sum', 'count']).reset_index()
first_down_prob['first_down_prob'] = first_down_prob['sum']/first_down_prob['count']

intervals = []
conditions = []

for i in range(len(first_down_prob)):
    intervals.append(first_down_prob['first_down_prob'][i])
    conditions.append( (dataframe["_down"] == first_down_prob["_down"][i]) & (dataframe["_distance"] == first_down_prob['_distance'][i]))

dataframe["first_down_prob"] = np.select(conditions, intervals)


In [18]:
# Now let's create the models for the usual 4th down decision making. I will make 3 different models (Field Goals, Go For It, and Punting)

subset = dataframe[['_play_type', '_down', '_distance', 'time_intervals', 'field_position_intervals', 'score_differential', 'first_down_prob', 'xPa']]
subset = subset[~subset['_play_type'].isin(['Kickoff', 'Uncategorized'])]

fourth_down = subset[subset['_down'] == 4]
fourth_down_dummies = pd.get_dummies(fourth_down, columns=['time_intervals', "field_position_intervals"])
predictiors = fourth_down_dummies.columns.drop(['xPa', '_play_type'])
prediction_set = fourth_down_dummies[predictiors]

The models will be split up by the 3 decisions that could be made. One model for Field Goals, one for punting, and one for going for it.
I will then use each model to predict the xPa value for each 4th down situation for each deicison.

In [8]:


from sklearn.model_selection import train_test_split
from utils import rf_regress_params_tuner


field_goals_dummies = fourth_down_dummies[fourth_down_dummies["_play_type"].isin(['Field Goal Good', 'Missed Field Goal Return Touchdown', 'Missed Field Goal Return', 'Blocked Field Goal', 'Field Goal Missed', 'Blocked Field Goal Touchdown'])]

predictors = field_goals_dummies.columns.drop(['xPa', '_play_type']) #predictor variables used: all variables besides the target variable
target = field_goals_dummies['xPa'].values #target variable


#splits the subset into a training set to fit the models on and a testing set to test the models on for their accuracy
fg_train_data, fg_test_data, fg_train_sln, fg_test_sln = train_test_split(field_goals_dummies[predictors], target, test_size = 0.2, random_state=0)

field_goals_params = rf_regress_params_tuner(fg_train_data, fg_test_data, fg_train_sln, fg_test_sln, field_goals_dummies)
print(field_goals_params)

INFO:root:Best r2 after tuning max depth is: 0.6641880287493502
INFO:root:Best r2 after tuning n_estimators is: 0.6711581175960506
INFO:root:Best r2 after tuning min samples split is: 0.6711581175960506
INFO:root:Best r2 after tuning min samples leaf is: 0.6711581175960506
INFO:root:Best r2 after tuning max features is: 0.6747634270684506
INFO:root:Final Model Accuracy 0.6747634270684506
INFO:root:Tuned parameters: {'max_depth': 29, 'n_estimators': 300, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 22}
INFO:root:Random Forest Feature Importance [0.00000000e+00 1.06201783e-01 2.20351885e-01 1.23306895e-01
 1.91109725e-02 3.07135265e-02 2.63237150e-02 1.03305859e-02
 2.16898430e-02 2.56722718e-02 1.65056277e-02 1.12776674e-02
 2.38360753e-02 1.06112432e-02 1.07282739e-02 1.67848659e-02
 1.02766776e-02 8.50697794e-03 8.68294782e-03 9.72719427e-03
 5.60454903e-03 4.50279806e-03 9.23264192e-03 2.99604922e-02
 0.00000000e+00 1.20942920e-02 1.81090693e-02 1.44006261e-02
 1.20

{'max_depth': 29, 'n_estimators': 300, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 22}


In [9]:
punts_dummies = fourth_down_dummies[fourth_down_dummies["_play_type"].isin(['Punt', 'Blocked Punt', 'Punt Return Touchdown', 'Blocked Punt Touchdown'])]


predictors = punts_dummies.columns.drop(['xPa', '_play_type']) #predictor variables used: all variables besides the target variable
target = punts_dummies['xPa'].values #target variable


#splits the subset into a training set to fit the models on and a testing set to test the models on for their accuracy
punts_train_data, punts_test_data, punts_train_sln, punts_test_sln = train_test_split(punts_dummies[predictors], target, test_size = 0.2, random_state=0)

punts_params = rf_regress_params_tuner(punts_train_data, punts_test_data, punts_train_sln, punts_test_sln, punts_dummies)

INFO:root:Best r2 after tuning max depth is: 0.6817087230473577
INFO:root:Best r2 after tuning n_estimators is: 0.6828974915995798
INFO:root:Best r2 after tuning min samples split is: 0.6828974915995798
INFO:root:Best r2 after tuning min samples leaf is: 0.6828974915995798
INFO:root:Best r2 after tuning max features is: 0.6891216816219041
INFO:root:Final Model Accuracy 0.6891216816219041
INFO:root:Tuned parameters: {'max_depth': 21, 'n_estimators': 150, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 34}
INFO:root:Random Forest Feature Importance [0.00000000e+00 1.74567772e-01 3.39776435e-01 1.49777283e-01
 1.25829797e-02 1.09451333e-02 1.47252159e-02 2.79968111e-03
 9.92548174e-03 8.25108781e-03 1.42775856e-02 1.99944620e-03
 4.50943865e-03 6.19521704e-03 9.76691337e-04 1.43458371e-02
 2.11333993e-03 5.18612841e-03 2.33588970e-04 3.99026571e-05
 1.69390113e-04 6.53665068e-05 0.00000000e+00 1.80248573e-02
 0.00000000e+00 1.66885451e-05 5.22096488e-03 7.53027956e-03
 6.33

In [10]:
go_for_it_dummies = fourth_down_dummies[~fourth_down_dummies["_play_type"].isin(['Field Goal Good', 'Missed Field Goal Return Touchdown', 'Missed Field Goal Return', 'Blocked Field Goal', 'Field Goal Missed', 'Blocked Field Goal Touchdown',
                                                            'Punt', 'Blocked Punt', 'Punt Return Touchdown', 'Blocked Punt Touchdown', 'Penalty', 'Timeout',  'Kickoff Return (Offense)', 'Kickoff Return Touchdown'])]


predictors = go_for_it_dummies.columns.drop(['xPa', '_play_type']) #predictor variables used: all variables besides the target variable
target = go_for_it_dummies['xPa'].values #target variable


#splits the subset into a training set to fit the models on and a testing set to test the models on for their accuracy
go_for_it_train_data, go_for_it_test_data, go_for_it_train_sln, go_for_it_test_sln = train_test_split(go_for_it_dummies[predictors], target, test_size = 0.2, random_state=0)

go_for_it_params = rf_regress_params_tuner(go_for_it_train_data, go_for_it_test_data, go_for_it_train_sln, go_for_it_test_sln, field_goals_dummies)

INFO:root:Best r2 after tuning max depth is: 0.5963175838062211
INFO:root:Best r2 after tuning n_estimators is: 0.6103379766426366
INFO:root:Best r2 after tuning min samples split is: 0.6103379766426366
INFO:root:Best r2 after tuning min samples leaf is: 0.6103379766426366
INFO:root:Best r2 after tuning max features is: 0.663248595704808
INFO:root:Final Model Accuracy 0.663248595704808
INFO:root:Tuned parameters: {'max_depth': 23, 'n_estimators': 650, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 8}
INFO:root:Random Forest Feature Importance [0.00000000e+00 1.03029444e-01 3.13419562e-01 1.08044458e-01
 1.08384087e-02 1.61741421e-02 1.97843704e-02 7.09008905e-03
 3.92364950e-02 2.00261946e-02 1.82238393e-02 1.76000322e-02
 2.46928058e-02 1.34976747e-02 7.03613373e-04 6.35831524e-04
 2.00956330e-05 1.76222262e-03 2.39814561e-03 1.89838905e-04
 3.90651166e-03 1.53572079e-02 1.58927470e-03 1.32249638e-02
 0.00000000e+00 1.19717287e-02 3.72738388e-04 1.55978422e-02
 1.29596

In [21]:
from sklearn.ensemble import RandomForestRegressor


fg_forest = RandomForestRegressor(
        max_depth=field_goals_params['max_depth'],
        n_estimators=field_goals_params['n_estimators'],
        min_samples_split=field_goals_params['min_samples_split'],
        min_samples_leaf=field_goals_params['min_samples_leaf'],
        max_features=field_goals_params['max_features'],
        random_state=0,
    )

fg_forest.fit(fg_train_data, fg_train_sln)
predictions = fg_forest.predict(prediction_set)

fourth_down["field_goal_xPa"] = predictions
fourth_down.head()

C:\Users\jordb\AppData\Local\Temp\ipykernel_14144\2883989057.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fourth_down["field_goal_xPa"] = predictions


,_play_type,_down,_distance,time_intervals,field_position_intervals,score_differential,first_down_prob,xPa,field_goal_xPa
10,Punt,4,3,1-4,51-55,0,0.407563,0.0,0.693794
33,Punt,4,6,7-10,26-30,7,0.342199,0.0,0.304135
64,Field Goal Missed,4,4,7-10,66-70,7,0.352641,0.0,0.125282
105,Punt,4,7,1-4,21-25,7,0.315556,0.0,0.075000
116,Punt,4,5,10-13,41-45,7,0.354812,0.0,0.231237


In [23]:
punt_forest = RandomForestRegressor(
        max_depth=punts_params['max_depth'],
        n_estimators=punts_params['n_estimators'],
        min_samples_split=punts_params['min_samples_split'],
        min_samples_leaf=punts_params['min_samples_leaf'],
        max_features=punts_params['max_features'],
        random_state=0,
    )

punt_forest.fit(punts_train_data, punts_train_sln)
predictions = punt_forest.predict(prediction_set)

fourth_down["punt_xPa"] = predictions

C:\Users\jordb\AppData\Local\Temp\ipykernel_14144\2068766211.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fourth_down["punt_xPa"] = predictions


In [24]:
gfi_forest = RandomForestRegressor(
        max_depth=go_for_it_params['max_depth'],
        n_estimators=go_for_it_params['n_estimators'],
        min_samples_split=go_for_it_params['min_samples_split'],
        min_samples_leaf=go_for_it_params['min_samples_leaf'],
        max_features=go_for_it_params['max_features'],
        random_state=0,
    )

gfi_forest.fit(go_for_it_train_data, go_for_it_train_sln)
predictions = gfi_forest.predict(prediction_set)

fourth_down["go_for_it_xPa"] = predictions

C:\Users\jordb\AppData\Local\Temp\ipykernel_14144\1588984049.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fourth_down["go_for_it_xPa"] = predictions


In [26]:
fourth_down.head()

,_play_type,_down,_distance,time_intervals,field_position_intervals,score_differential,first_down_prob,xPa,field_goal_xPa,punt_xPa,go_for_it_xPa
10,Punt,4,3,1-4,51-55,0,0.407563,0.0,0.693794,0.00,-0.002859
33,Punt,4,6,7-10,26-30,7,0.342199,0.0,0.304135,0.00,-0.185619
64,Field Goal Missed,4,4,7-10,66-70,7,0.352641,0.0,0.125282,-0.22,-1.716093
105,Punt,4,7,1-4,21-25,7,0.315556,0.0,0.075000,0.00,0.336734
116,Punt,4,5,10-13,41-45,7,0.354812,0.0,0.231237,-0.14,-0.509150


In [41]:
conditions = [
    (fourth_down['field_goal_xPa'] > fourth_down['punt_xPa']) & (fourth_down['field_goal_xPa'] > fourth_down['go_for_it_xPa']),
    (fourth_down['punt_xPa'] > fourth_down['field_goal_xPa']) & (fourth_down['punt_xPa'] > fourth_down['go_for_it_xPa']),
    (fourth_down['go_for_it_xPa'] > fourth_down['field_goal_xPa']) & (fourth_down['go_for_it_xPa'] > fourth_down['punt_xPa'])
]

values = ['Attempt FG', 'Punt', 'Go For It']

fourth_down["suggested_decision"] = np.select(conditions, values)

fourth_down.to_excel('fourth_down_decision_making.xlsx', index=False)
